In [1]:
# External libraries
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [76]:
# Loading All Images:
dict = load_all_images(dt=10)
# Img:
idx = 0  # Loading image n. idx+1 ...
filename = dict[str(idx+1)]
img=cv2.imread(filename)
# Detection:
t1 = time.time()
craters_det = detect(img)
t2 = time.time()
print(f'Detection Time:{t2-t1:.2f}\n')
# Pandas DataFrame:
df_craters_det = sort_mat(craters_det)
# Find all triplets:
t1 = time.time()
triplets = find_all_triplets(craters_det)
triplets_det= pd.DataFrame(triplets, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
triplets_det.shape
t2 = time.time()
print(f'Number of total combinations:{triplets_det.shape[0]}\nComputational time: {t2-t1:.2f} s')

Detection Time:11.78

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Number of total combinations:7980
Computational time: 0.25 s


In [86]:
# Opening Database:
DB = pd.read_csv('DATA/H_L_combined.csv')
# DB = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')
# Filtering:
craters_cat = CatalogSearch(DB, lat_bounds=[-1.5, 1.5], lon_bounds=[-147.81-1.5,-147.81+1.5], CAT_NAME='COMBINED')

km2deg = 1/deg2km
craters_cat = craters_cat[(craters_cat.Diam < 40)&(craters_cat.Diam > 2.5)]
craters_cat['Diam']*=0.5*km2deg # km --- > deg

craters_cat_m = np.array(craters_cat)
t1 = time.time()
triplets_cat_m = find_all_triplets(craters_cat_m)
triplets_cat = pd.DataFrame(triplets_cat_m, columns=['Angle1','Angle2','Angle3','des1','des2','des3','lon1','lat1','r1','lon2','lat2','r2','lon3','lat3','r3'])
t2 = time.time()
print(f'Number of total combinations:{triplets_cat.shape[0]}\nComputational time: {t2-t1:.2f} s')

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Number of total combinations:210
Computational time: 0.01 s


In [87]:
lat_bounds=[-1.5, 1.5]
lon_bounds=[-147.81-1.5,-147.81+1.5]
#img1
cp1 = deepcopy(img)
img_det = img_plus_crts(img, craters_det)
plt.figure()
plt.subplot(121)
plt.imshow(img_det)
plt.show()

# FIG.2
cp1 = deepcopy(img)
# DB = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')
DB = pd.read_csv('DATA/H_L_combined.csv')
df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME='COMBINED')
df = df[df.Diam > 2]
image_with_craters = draw_craters_on_image(df,  lon_bounds, lat_bounds, cp1, u=None)

plt.subplot(122)
plt.imshow(image_with_craters)
plt.show()

In [89]:
t1 = time.time()
QUERY1 = triplets_cat
QUERY2 = triplets_det
QUERY1 = dropduplicates(QUERY1)
QUERY2 = dropduplicates(QUERY2) 
QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)
QUERY2 =QUERY2[ (QUERY2.Angle1 > 10) & (QUERY2.Angle2 > 10) & (QUERY2.Angle3 > 10) ].reset_index(drop=True)
if QUERY1.shape[0]<QUERY2.shape[0]:
    joins, items = inner_join(QUERY1, QUERY2, 5)
else:
    joins, items = inner_join(QUERY2, QUERY1, 5)
t2 = time.time()
print(f'Computational time: {t2-t1:.2f} s\nPossible Combinations: {len(items)}')

Computational time: 0.23 s
Possible Combinations: 25


In [92]:
joins[0].head(3)

,Angle1,Angle2,Angle3,des1,des2,des3,x1,y1,r1,x2,y2,r2,x3,y3,r3
1,28.021957,36.009859,115.968184,23.499119,18.072029,8.686520,134.5,27.5,22.5,794.5,761.5,26.5,290.0,654.0,24.00
108,29.033494,40.442460,110.524046,2.939210,10.768192,8.924158,134.5,27.5,22.5,83.0,164.0,12.5,329.5,28.0,17.25
130,34.054551,38.633446,107.312003,20.696621,23.252811,6.931897,134.5,27.5,22.5,500.5,829.5,19.0,626.5,328.0,31.25


In [91]:
items[0]

Angle1     32.219838
Angle2     36.105308
Angle3    111.674854
des1        2.317715
des2        6.615749
des3        4.291510
lon1     -146.567992
lat1       -1.136915
r1          0.615915
lon2     -148.862726
lat2        0.327445
r2          0.206019
lon3     -147.303944
lat3        0.424471
r3          0.143797
Name: 0, dtype: float64

In [93]:
t1 = time.time()
tol2 = 1
S, iss = [], []
for i in range(len(joins)):
    join = joins[i]
    des1, des2, des3 = items[i].des1, items[i].des2, items[i].des3
    s=join[ (abs(join.des1 - des1) < tol2) & (abs(join.des2 - des2) < tol2) & (abs(join.des3 - des3) < tol2)\
          | (abs(join.des1 - des2) < tol2) & (abs(join.des2 - des3) < tol2) & (abs(join.des3 - des1) < tol2)\
          | (abs(join.des1 - des3) < tol2) & (abs(join.des2 - des1) < tol2) & (abs(join.des3 - des2) < tol2)]

    if s.shape[0] > 0:
        S.append(s)
        iss.append(i)
t2 = time.time()
print(f'Computational time: {t2-t1:.2f} s\nPossible Combinations: {len(S)}')

Computational time: 0.17 s
Possible Combinations: 2


In [94]:
print('Combinations are:')
S[0]

Combinations are:


,Angle1,Angle2,Angle3,des1,des2,des3,x1,y1,r1,x2,y2,r2,x3,y3,r3
442,33.053197,33.173022,113.77378,6.399295,4.845535,3.183011,547.5,549.5,15.5,506.0,739.0,20.5,465.0,630.5,13.25


In [95]:
print('With:')
items[iss[0]]

With:


Angle1     32.219838
Angle2     36.105308
Angle3    111.674854
des1        2.317715
des2        6.615749
des3        4.291510
lon1     -146.567992
lat1       -1.136915
r1          0.615915
lon2     -148.862726
lat2        0.327445
r2          0.206019
lon3     -147.303944
lat3        0.424471
r3          0.143797
Name: 0, dtype: float64

In [100]:
# Computing centroid local:
hp=S[0]
x1,x2,x3 = hp.x1,hp.x2,hp.x3
y1,y2,y3 = hp.y1,hp.y2,hp.y3
xc = (x1+x2+x3)/3
yc = (y1+y2+y3)/3

XC, YC = 850/2, 850/2

Delta = np.array([XC-xc, YC-yc])

# Recompute centroid in LAT-LON:
hp = items[iss[0]]
x1,x2,x3 = hp.lon1,hp.lon2,hp.lon3
y1,y2,y3 = hp.lat1,hp.lat2,hp.lat3
xc_lon = (x1+x2+x3)/3
yc_lat = (y1+y2+y3)/3

u = 257.52  # ? DEG TO PXS
px2deg = 1/u
Delta*= px2deg
pos = [xc_lon-Delta[0], yc_lat-Delta[1]]
print(f'Position  is:      LON  {pos[0]},         LAT  {pos[1]}')

Position  is:      LON  [-147.26303484],         LAT  [0.70525924]


In [101]:
Delta

array([[-0.31518588],
       [-0.83359221]])

In [ ]:
if len(joins)>1:
    print(len(joins))
    plt.close()
    i = 6
    arg = items[i][0]
    a = QUERY1.iloc[arg]
    crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
    
    plt.figure(dpi=200)
    plt.subplot(121)
    cp1 = img.copy()
    IMG1 =  img_plus_crts(cp1, crts1, color="red")
    plt.imshow(IMG1)
    plt.show()

    join = joins[i]
    j=1
    b = join.iloc[j]
    crts2 = np.vstack([[b.lon1,b.lat1,b.r1],[b.lon2,b.lat2,b.r2],[b.lon3,b.lat3,b.r3]])
    crts2[:,2]*= deg2km
    plt.subplot(122)
    lat_bounds=[-1.5, 1.5]
    lon_bounds=[-129.802-1.5,-129.802+1.5]
    crts2_to_rel = craters_to_relative_frame(crts2, lon_b=lon_bounds, lat_b=lat_bounds)
    IMG2 =  img_plus_crts(cp1, crts2_to_rel, color="green")
    plt.imshow(IMG2)
    plt.show()

In [ ]:
c2 = compute_centroid(crts2)
c1 = compute_centroid(crts1)

In [ ]:
def craters_to_relative_frame(df, lon_b, lat_b, u=None):
    lon_bounds = lon_b
    lat_bounds = lat_b
    # CAMERA CENTER:
    CAMx, CAMy = (
        (lon_bounds[0] + lon_bounds[1]) / 2,
        (lat_bounds[0] + lat_bounds[1]) / 2,
    )

    if u == None:  # Scale Factor
        u = 257.52  # ? DEG TO PXS
        span = (abs(lon_b[0]) - abs(lon_b[1])) * u
        span = abs(int(span))
        print(span)
    else:
        span = (abs(lon_b[0]) - abs(lon_b[1])) * u
        span = abs(int(span))
    # Make the img:
    # img = np.zeros((span, span), dtype=int)
    if df is None:
        print("No crater found")
        pass
    else:
        W, H = (span, span)
        # Cycle through the dataframe:
        craters = np.zeros(3)
        for i in range(df.shape[0]):
            crater = df[i]
            # crater center:
            xc, yc = crater[0], crater[1]  # This is in the absolute frame
            # f: Absolute --> f: Relative
            xc = xc - CAMx
            yc = yc - CAMy
            # f: relative --> f: OPENCV
            xc *= u  # Now is in pixel not in lon deg
            yc *= u  # Now is in pixel not in lat deg
            xc = W / 2 + xc
            yc = H / 2 - yc
            # ? 1 km = 8.4746 px in our DEM := Merge LOLA - KAGUYA
            KM_to_PX = 1/0.118
            crater_i = [xc, yc, crater[2] * KM_to_PX]
            craters = np.vstack([craters, crater_i])
        return craters[1:, :]